In [1]:
import matplotlib.pyplot as plt
import numpy as np
from gym.wrappers import (
    FilterObservation,
    FlattenObservation,
    RecordVideo,
    RescaleAction,
    TimeLimit,
)
from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.env_util import unwrap_wrapper

from ea_train import ARESEACheetah


initializing ocelot...


In [2]:
env = ARESEACheetah(
    incoming_mode="random",
    incoming_values=None,
    misalignment_mode="random",
    misalignment_values=None,
    action_mode="direct",
    magnet_init_mode="random",
    magnet_init_values=None,
    reward_mode="differential",
    target_beam_mode="random",
    target_beam_values=None,
    w_mu_x=1.0,
    w_mu_y=1.0,
    w_on_screen=1.0,
    w_sigma_x=1.0,
    w_sigma_y=1.0,
    w_time=1.0,
)
env = RescaleAction(env, -1, 1)

check_env(env)


In [3]:
env.reset()

{'beam': array([-0.00195788,  0.00217555,  0.01382812,  0.00282404], dtype=float32),
 'magnets': array([-3.4112434e+01,  2.1074467e+01, -6.0631298e-03, -5.2898746e+01,
         5.5924780e-03], dtype=float32),
 'target': array([-1.1523461e-04,  4.5870886e-05, -1.9672473e-03,  7.6189573e-04],
       dtype=float32),
 'incoming': array([ 1.4000757e+08, -5.4798590e-04,  4.9217608e-05,  8.8691537e-04,
        -5.2436350e-05,  4.1198116e-04,  4.4543332e-05,  2.8576390e-04,
         2.8824101e-05,  2.1663573e-06,  3.2638101e-04], dtype=float32),
 'misalignments': array([ 9.3962635e-06, -4.4789066e-04, -4.5861621e-04, -3.1727768e-04,
        -4.8978598e-04, -3.2253590e-04, -3.7911246e-04,  4.6150855e-04],
       dtype=float32)}

In [4]:
env.step(np.zeros(5))

({'beam': array([-6.8172259e-05,  4.2194137e-04,  3.1811997e-04,  2.9178601e-04],
        dtype=float32),
  'magnets': array([0., 0., 0., 0., 0.], dtype=float32),
  'target': array([-1.1523461e-04,  4.5870886e-05, -1.9672473e-03,  7.6189573e-04],
        dtype=float32),
  'incoming': array([ 1.4000757e+08, -5.4798590e-04,  4.9217608e-05,  8.8691537e-04,
         -5.2436350e-05,  4.1198116e-04,  4.4543332e-05,  2.8576390e-04,
          2.8824101e-05,  2.1663573e-06,  3.2638101e-04], dtype=float32),
  'misalignments': array([ 9.3962635e-06, -4.4789066e-04, -4.5861621e-04, -3.1727768e-04,
         -4.8978598e-04, -3.2253590e-04, -3.7911246e-04,  4.6150855e-04],
        dtype=float32)},
 0.8543914853899783,
 False,
 {'binning': array(4),
  'l1_distance': 0.0031786098661541473,
  'on_screen_reward': 1,
  'pixel_size': array([1.32792e-05, 9.78760e-06]),
  'screen_resolution': array([612., 510.]),
  'time_reward': -1})